**<font color=red>看看数据长什么样</font>**<br>
还是用pandas加载数据

In [14]:
# 这个ipython notebook主要是我解决Kaggle Titanic问题的思路和过程
#!/usr/bin/python
# -*- coding: UTF-8 -*-

import pandas as pd #数据分析
import numpy as np #科学计算
from pandas import Series,DataFrame

import matplotlib

import matplotlib.pyplot as plt
#有中文出现的情况，需要u'内容'

data_train = pd.read_excel("Resource/1.xlsx")
data_train.columns
#data_train[data_train.Cabin.notnull()]['Survived'].value_counts()

Index([u'TH', u'U', u'K', u'ANS'], dtype='object')

**<font color=red>我们看大概有以下这些字段</font>**<br>
TH => <br>
U => <br>
K => <br>
ans =>  <br>

**<font color=red>让pandas自己先告诉我们一些信息<font>**

In [15]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
TH     27 non-null float64
U      27 non-null float64
K      27 non-null float64
ANS    27 non-null float64
dtypes: float64(4)
memory usage: 936.0 bytes


In [16]:
data_train.describe()

,TH,U,K,ANS
count,27.000000,27.000000,27.000000,27.000000
mean,6.375667,8.018037,5.996630,96.118519
std,3.679972,23.209275,9.870309,228.373402
min,2.589000,0.633000,1.704000,20.600000
25%,3.865000,1.345000,2.416000,39.440000
50%,5.160000,1.902000,2.927000,50.700000
75%,7.480000,2.705500,3.322000,69.050000
max,17.127000,118.950000,47.145000,1234.900000


**<font color=red>用逻辑回归计算<font>**<br>
<font color=red>我们把需要的feature字段取出来，转成numpy格式，使用scikit-learn中的LogisticRegression建模。<font>

In [26]:
# 我们把需要的feature字段取出来，转成numpy格式，使用scikit-learn中的LogisticRegression建模
from sklearn import linear_model
dummies_TH = pd.get_dummies(data_train['TH'], prefix= 'TH')
dummies_U = pd.get_dummies(data_train['U'], prefix= 'U')
dummies_K = pd.get_dummies(data_train['K'], prefix= 'K')
dummies_ANS = pd.get_dummies(data_train['ANS'], prefix= 'ANS')
df = pd.concat([data_train, dummies_TH, dummies_U, dummies_K], axis=1)
df.drop(['ANS','TH', 'U', 'K'], axis=1, inplace=True)

train_df = df.filter(regex='ANS,TH|U|K')
train_np = train_df.as_matrix()

# y即Survival结果
y = train_np[:, 0]

# X即特征属性值
X = train_np[:, 1:]

# fit到RandomForestRegressor之中
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
clf.fit(X, y)
clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [22]:
X.shape

(27L, 53L)

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.learning_curve import learning_curve

# 用sklearn的learning_curve得到training_score和cv_score，使用matplotlib画出learning curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=1, 
                        train_sizes=np.linspace(.05, 1., 20), verbose=0, plot=True):
    """
    画出data在某模型上的learning curve.
    参数解释
    ----------
    estimator : 你用的分类器。
    title : 表格的标题。
    X : 输入的feature，numpy类型
    y : 输入的target vector
    ylim : tuple格式的(ymin, ymax), 设定图像中纵坐标的最低点和最高点
    cv : 做cross-validation的时候，数据分成的份数，其中一份作为cv集，其余n-1份作为training(默认为3份)
    n_jobs : 并行的的任务数(默认1)
    """
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, verbose=verbose)
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    if plot:
        plt.figure()
        plt.title(title)
        if ylim is not None:
            plt.ylim(*ylim)
        plt.xlabel(u"训练样本数")
        plt.ylabel(u"得分")
        plt.gca().invert_yaxis()
        plt.grid()
    
        plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, 
                         alpha=0.1, color="b")
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, 
                         alpha=0.1, color="r")
        plt.plot(train_sizes, train_scores_mean, 'o-', color="b", label=u"训练集上得分")
        plt.plot(train_sizes, test_scores_mean, 'o-', color="r", label=u"交叉验证集上得分")
    
        plt.legend(loc="best")
        
        plt.draw()
        plt.gca().invert_yaxis()
        plt.show()
    
    midpoint = ((train_scores_mean[-1] + train_scores_std[-1]) + (test_scores_mean[-1] - test_scores_std[-1])) / 2
    diff = (train_scores_mean[-1] + train_scores_std[-1]) - (test_scores_mean[-1] - test_scores_std[-1])
    return midpoint, diff

plot_learning_curve(clf, u"学习曲线", X, y)

E:\Soft\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
E:\Soft\Anaconda2\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
E:\Soft\Anaconda2\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)
E:\Soft\Anaconda2\lib\site-packages\sklearn\le

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0